## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Palmer,US,42.1584,-72.3287,21.76,42,0,5.75,clear sky
1,1,Aloleng,PH,16.1307,119.7824,76.53,78,3,5.99,clear sky
2,2,Lithgow,AU,-33.4833,150.1500,55.71,91,64,6.49,broken clouds
3,3,Fresnillo,MX,23.1667,-102.8833,56.75,35,95,17.85,overcast clouds
4,4,Krasnoselkup,RU,65.7000,82.4667,2.70,98,76,10.45,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temperature = float(input("Please enter the minimum temperature?"))
max_temperature = float(input("Please enter the maximum temperature?"))

Please enter the minimum temperature?70
Please enter the maximum temperature?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_data_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temperature) & (city_data_df['Max Temp'] <= max_temperature)]

In [5]:
# 4a. Determine if there are any empty rows.
filtered_city_data_df.isna().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_data_df2 = filtered_city_data_df.dropna()


filtered_city_data_df2.isna().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_city_data_df2[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Aloleng,PH,76.53,clear sky,16.1307,119.7824,
5,Rikitea,PF,78.44,overcast clouds,-23.1203,-134.9692,
6,Hilo,US,78.80,clear sky,19.7297,-155.0900,
10,Port Alfred,ZA,74.91,few clouds,-33.5906,26.8910,
11,Busselton,AU,73.04,broken clouds,-33.6500,115.3333,
12,Tucurui,BR,80.78,scattered clouds,-3.7661,-49.6725,
15,Saint-Philippe,RE,80.19,light rain,-21.3585,55.7679,
16,Bandarbeyla,SO,76.37,overcast clouds,9.4942,50.8122,
17,Bredasdorp,ZA,72.28,broken clouds,-34.5322,20.0403,
20,Prieska,ZA,77.16,overcast clouds,-29.6641,22.7474,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i in hotel_df.index:
    # 6c. Get latitude and longitude from DataFrame
    Lat = hotel_df['Lat'][i]
    Lng = hotel_df['Lng'][i]
    params["location"] = f"{Lat},{Lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    #print(hotels["results"][0]["name"])
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df_final = hotel_df.dropna()
hotel_df_final = hotel_df_final[hotel_df_final['Hotel Name']!='']

In [10]:
hotel_df_final

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Aloleng,PH,76.53,clear sky,16.1307,119.7824,Steve Walden and Emma Sison-Walden Residence
5,Rikitea,PF,78.44,overcast clouds,-23.1203,-134.9692,People ThankYou
6,Hilo,US,78.80,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Port Alfred,ZA,74.91,few clouds,-33.5906,26.8910,The Halyards Hotel
11,Busselton,AU,73.04,broken clouds,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
553,Kita,ML,76.71,overcast clouds,13.0349,-9.4895,Hôtel Chat Rouge
555,Ngunguru,NZ,73.17,few clouds,-35.6167,174.5000,Pacific Rendezvous Motel Resort
562,Domoni,KM,79.20,overcast clouds,-12.2569,44.5319,Al Djadid
563,Clarence Town,AU,73.85,broken clouds,-32.5833,151.7833,River Downs Cottage


In [11]:
# 8a. Create the output File (CSV)
output = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df_final.to_csv(output)

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_final.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_final[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
!jupyter nbextension enable --py gmaps
